In [1]:
import sys
sys.path.append("..")

import h5py
import numpy as np
from rich.progress import Progress
from rich.console import Console
from hazma.parameters import (
    electron_mass as me,
    muon_mass as mmu,
    charged_pion_mass as mpi
)

from gecko import SingleChannelConstraints
from gecko import HiggsPortalConstraints
from gecko import KineticMixingConstraints
from gecko.utils import add_to_dataset

## Single-Channel

In [3]:
singlechannel = SingleChannelConstraints()
masses = {
    'e e': np.geomspace(me, 1e4, 100),
    'mu mu': np.geomspace(mmu, 1e4, 100),
    'g g': np.geomspace(1e-1, 1e4, 100),
    'pi pi': np.geomspace(mpi, 1e4, 100)
}

In [4]:
ann_constraints = {}
dec_constraints = {}
with Progress(transient=True) as progress:
    for fs, mxs in masses.items():
        ann_constraints[fs] = singlechannel.compute(mxs, fs, decay=False, progress=progress)
with Progress(transient=True) as progress:
    for fs, mxs in masses.items():
        dec_constraints[fs] = singlechannel.compute(mxs, fs, decay=True, progress=progress)

Output()

Output()

This uses \(N_{\mathrm{S}}/\sqrt{N_{B}} > 25\) for a discovery. This is used as an estimate for how the constraints change if the background for GECCO is larger by a factor of 10 ish larger then we'd expect.

In [8]:
ann_constraints_25sigma = {}
dec_constraints_25sigma = {}
kwargs = {"existing": False, "cmb": False, "sigma": 25.0}
with Progress(transient=True) as progress:
    for fs, mxs in masses.items():
        ann_constraints_25sigma[fs] = singlechannel.compute(mxs, fs, decay=False, progress=progress, **kwargs)
with Progress(transient=True) as progress:
    for fs, mxs in masses.items():
        dec_constraints_25sigma[fs] = singlechannel.compute(mxs, fs, decay=True, progress=progress, **kwargs)

Output()

Output()

In [40]:
with h5py.File("../data/single_channel_ann.hdf5", "w") as dataset:
    add_to_dataset(dataset, ann_dec_constraints['ann'])
    
with h5py.File("../data/single_channel_dec.hdf5", "w") as dataset:
    add_to_dataset(dataset, ann_dec_constraints['dec'])

## Higgs Portal

In [2]:
hp = HiggsPortalConstraints()
mxs = np.geomspace(0.1, 100.0, 20)
hp_constraints = {}
with Progress(transient=True) as progress:
    hp_constraints = hp.compute(mxs, 0.5, progress=progress, gecco=False)


Output()

In [3]:
hp = HiggsPortalConstraints()
mxs = np.geomspace(0.1, 250.0, 100)
hp_constraints = {}

with Progress(transient=True) as progress:
    hp_constraints["ms/mx=1.5"] = hp.compute(mxs, 2.0, progress=progress)
with Progress(transient=True) as progress:
    hp_constraints["ms/mx=0.5"] = hp.compute(mxs, 0.5, progress=progress)

Output()

Output()

In [6]:
hp_constraints_25sigma = {}
includes = {"include_existing": False, "include_cmb": False, "include_pheno": False, "include_relic_density": False}
with Progress(transient=True) as progress:
    hp_constraints_25sigma["ms/mx=2"] = hp.compute(mxs, 2.0, progress=progress, sigma=25, **includes)
with Progress(transient=True) as progress:
    hp_constraints_25sigma["ms/mx=0.5"] = hp.compute(mxs, 0.5, progress=progress, sigma=25, **includes)

Output()

Output()

## Kinetic Mixing

In [4]:
km = KineticMixingConstraints()
mxs = np.geomspace(0.1, 250.0, 100)
km_constraints = {}
with Progress(transient=True) as progress:
    km_constraints = km.compute(mxs, 3.0, progress=progress)

Output()

In [5]:
km_constraints

{'cmb': array([           inf,            inf,            inf,            inf,
                   inf,            inf,            inf,            inf,
                   inf,            inf,            inf,            inf,
                   inf,            inf,            inf,            inf,
                   inf,            inf,            inf,            inf,
                   inf, 1.11103953e-30, 1.24321869e-30, 1.39652501e-30,
        1.48463918e-30, 1.56791816e-30, 1.65367153e-30, 1.74177346e-30,
        1.83206882e-30, 1.92437013e-30, 2.00258266e-30, 1.92802262e-30,
        1.86399367e-30, 1.80857604e-30, 1.76027910e-30, 1.71793481e-30,
        1.68061346e-30, 1.66579201e-30, 1.66951047e-30, 1.67297338e-30,
        1.67619081e-30, 1.67917470e-30, 1.68193727e-30, 1.68449261e-30,
        1.73952765e-30, 1.81995486e-30, 1.90116965e-30, 1.98292562e-30,
        2.06497097e-30, 2.14704881e-30, 2.22890196e-30, 2.36487904e-30,
        2.51623972e-30, 2.67440653e-30, 2.83932533e-30, 3

## RH-Neutrino